------
----
----

# SE-Residual

**Residual bloğun içindeki feature’ları kanal bazında yeniden ağırlıklandıran bir
attention ekidir.**

**SE-Residual yapısında, kanal attention mekanizması
residual bloğun dönüşüm fonksiyonu F(x) içine yerleştirilir.
SE, F(x)’in kanal çıktıları için önem katsayıları üretir
ve bu çıktılar residual bağlantı ile ana akışa eklenir.**

----
----
-----

# Squeeze-and-Excitation (SE) Residual Block — Temelden Derine

Bu notebook, **SE (Squeeze-and-Excitation)** fikrini sıfırdan alıp **Residual blok içine nasıl yerleştiğine** kadar götürür.

İçerik akışı:
1. CNN’lerde kanal önemlendirme problemi
2. SE modülünün mantığı (Squeeze → Excitation → Reweight)
3. Matematiksel sezgi + boyut takibi
4. PyTorch ile: `SEBlock`, `SEBasicBlock`, `SEBottleneck`
5. Nerede/niye kullanılır, pratik ayarlar ve sık hatalar


## 1) Problem: Convolution kanal önemini doğrudan ayırt etmez

Bir conv katmanının çıktısı genelde şu şekildedir:

- **X** ∈ ℝ^(B×C×H×W)

Burada:
- **C**: kanal sayısı (feature map sayısı)
- **H, W**: uzamsal boyutlar

Conv katmanı uzamsal desenleri iyi yakalar ama “bu kanalı aç, bunu kıs” işini **explicit** yapmaz.
İşte SE, **channel-wise attention** ile bu açığı kapatır.

Kısa özet:
- CNN: “nerede ne var” (spatial) güçlü
- SE: “hangi kanal önemli” (channel) net


## 2) SE fikri tek cümleyle

**SE şunu yapar:**
> Feature map’lere bakar, her kanal için 0–1 arası bir ağırlık üretir ve önemli kanalları güçlendirir.

SE’nin üç adımı var:
1. **Squeeze**: (H×W) bilgisini tek sayıya indir (kanal başına özet)
2. **Excitation**: Bu özetten kanal ağırlıkları öğren
3. **Reweight**: Ağırlıkları feature map ile çarp


## 3) Squeeze: Global Average Pooling ile kanal özeti çıkarma

Elimizde bir kanal olsun: X_c ∈ ℝ^(H×W)

Squeeze, her kanal için:
- `z_c = 평균(X_c)` üretir

Böylece:
- **z** ∈ ℝ^C

Bu vektör şuna benzer:
- “Her kanal genel olarak ne kadar aktif?”


## 4) Excitation: Küçük bir MLP ile kanal ağırlıkları üretme

Squeeze sonucu gelen **z ∈ ℝ^C** vektöründen, kanal ağırlıkları **s ∈ (0,1)^C** üretilir.

Tipik yapı:
- FC: C → C/r
- ReLU
- FC: C/r → C
- Sigmoid

Burada **r** reduction ratio (çoğunlukla 16) — hem parametreyi düşürür, hem nonlinearity sağlar.


## 5) Reweight: Kanal ağırlıklarını feature map’e uygulama

Kanal ağırlıkları:
- **s** ∈ ℝ^C

Feature map:
- **X** ∈ ℝ^(B×C×H×W)

Broadcast ile:
- `Y[b, c, h, w] = X[b, c, h, w] * s[b, c]`

Sonuç:
- Kanal bazında “gain” uygulamış olursun.


## 6) SE Residual Block: Nereye takılır?

Residual blok mantığı:
- `out = F(x) + shortcut(x)`

SE genelde **F(x)** yani residual branch’in **sonuna** konur:

- `out = SE(F(x)) + shortcut(x)`

Shortcut path çoğunlukla **dokunulmaz**, çünkü skip connection’ın temiz akması istenir.


## 7) PyTorch implementasyonu (notluk, anlaşılır)

Aşağıdaki kodlar:
- `SEBlock`: saf SE modülü
- `SEBasicBlock`: ResNet-18/34 tarzı basic residual + SE
- `SEBottleneck`: ResNet-50/101 tarzı bottleneck + SE


In [1]:
import torch
import torch.nn as nn
from typing import Optional

def conv3x3(in_ch: int, out_ch: int, stride: int = 1, groups: int = 1) -> nn.Conv2d:
    return nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=stride, padding=1, bias=False, groups=groups)

def conv1x1(in_ch: int, out_ch: int, stride: int = 1) -> nn.Conv2d:
    return nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride, padding=0, bias=False)

class SEBlock(nn.Module):
    def __init__(self, channels: int, reduction: int = 16):
        super().__init__()
        hidden = max(1, channels // reduction)

        # Squeeze: Global Average Pooling
        self.pool = nn.AdaptiveAvgPool2d(1)  # (B, C, 1, 1)

        # Excitation: küçük MLP (1x1 conv ile FC gibi davranır)
        self.fc1 = nn.Conv2d(channels, hidden, kernel_size=1, bias=True)
        self.act = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(hidden, channels, kernel_size=1, bias=True)
        self.gate = nn.Sigmoid()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # (B, C, H, W) -> (B, C, 1, 1)
        z = self.pool(x)

        # (B, C, 1, 1) -> (B, hidden, 1, 1) -> (B, C, 1, 1)
        s = self.fc1(z)
        s = self.act(s)
        s = self.fc2(s)
        s = self.gate(s)   # 0..1 arası kanal ağırlıkları

        # Reweight: broadcast çarpımı
        return x * s

### 7.1) SE + Basic Residual (ResNet-18/34 tipi)

- Residual branch: `Conv-BN-ReLU-Conv-BN`
- Sonuna SE ekle
- Topla + ReLU (post-act tarzı)


In [2]:
class SEBasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_ch: int, out_ch: int, stride: int = 1, se_reduction: int = 16):
        super().__init__()
        self.conv1 = conv3x3(in_ch, out_ch, stride=stride)
        self.bn1   = nn.BatchNorm2d(out_ch)
        self.relu  = nn.ReLU(inplace=True)

        self.conv2 = conv3x3(out_ch, out_ch, stride=1)
        self.bn2   = nn.BatchNorm2d(out_ch)

        self.se = SEBlock(out_ch, reduction=se_reduction)

        self.downsample = None
        if stride != 1 or in_ch != out_ch:
            self.downsample = nn.Sequential(
                conv1x1(in_ch, out_ch, stride=stride),
                nn.BatchNorm2d(out_ch),
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # SE residual branch'e uygulanır
        out = self.se(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out = out + identity
        out = self.relu(out)
        return out

### 7.2) SE + Bottleneck (ResNet-50/101 tipi)

Bottleneck yapısı:
- 1×1 reduce
- 3×3 process
- 1×1 expand (expansion genelde 4)

SE genelde **expand’den sonra** (yani residual branch çıkışında) uygulanır.


In [3]:
class SEBottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_ch: int, bottleneck_ch: int, stride: int = 1, se_reduction: int = 16):
        super().__init__()
        out_ch = bottleneck_ch * self.expansion

        self.conv1 = conv1x1(in_ch, bottleneck_ch, stride=1)
        self.bn1   = nn.BatchNorm2d(bottleneck_ch)

        self.conv2 = conv3x3(bottleneck_ch, bottleneck_ch, stride=stride)
        self.bn2   = nn.BatchNorm2d(bottleneck_ch)

        self.conv3 = conv1x1(bottleneck_ch, out_ch, stride=1)
        self.bn3   = nn.BatchNorm2d(out_ch)

        self.relu  = nn.ReLU(inplace=True)

        self.se = SEBlock(out_ch, reduction=se_reduction)

        self.downsample = None
        if stride != 1 or in_ch != out_ch:
            self.downsample = nn.Sequential(
                conv1x1(in_ch, out_ch, stride=stride),
                nn.BatchNorm2d(out_ch),
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.conv1(x); out = self.bn1(out); out = self.relu(out)
        out = self.conv2(out); out = self.bn2(out); out = self.relu(out)
        out = self.conv3(out); out = self.bn3(out)

        # SE residual output'ta (yani bottleneck sonunda)
        out = self.se(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out = out + identity
        out = self.relu(out)
        return out

## 8) Boyut kontrolü (sanity check)

Aşağıdaki hücre:
- SE modülünün boyutları koruduğunu
- Basic/Bottleneck SE blokların forward çalıştığını gösterir


In [4]:
x = torch.randn(2, 64, 56, 56)

se = SEBlock(64, reduction=16)
y = se(x)

b1 = SEBasicBlock(64, 64, stride=1, se_reduction=16)
y1 = b1(x)

b2 = SEBottleneck(64, bottleneck_ch=64, stride=1, se_reduction=16)  # output: 256 ch
y2 = b2(x)

(y.shape, y1.shape, y2.shape)

(torch.Size([2, 64, 56, 56]),
 torch.Size([2, 64, 56, 56]),
 torch.Size([2, 256, 56, 56]))

## 9) Pratik Notlar (gerçek hayatta işine yarayacak)

### 9.1 Reduction ratio (r) nasıl seçilir?
- Yaygın: **16**
- Küçük kanallarda (C küçükse): **8** daha mantıklı olabilir
- Çok küçük C’de `C//r` 0 olmasın diye `hidden=max(1, C//r)` yapılır (yukarıda yaptık)

### 9.2 SE nerede en çok işe yarar?
- Derin ağlar ve geniş kanal sayıları olan backbone’larda
- Özellikle sınıflandırma ve detection backbone’larında (ResNet/ResNeXt türevleri)

### 9.3 En sık yapılan hata
- SE’yi shortcut’a uygulamak (genelde gereksiz / bazen zararlı)
- Sıralamayı karıştırmak: SE, residual branch çıkışına uygulanmalı (çoğu tasarımda)

### 9.4 “SE = attention mı?”
Evet; ama **channel attention**.
CBAM gibi modüller bunun üzerine **spatial attention** da ekler.


## 10) Kısa özet

- **Squeeze:** GlobalAvgPool → kanal özet vektörü
- **Excitation:** küçük MLP → kanal ağırlıkları (0..1)
- **Reweight:** feature map kanallarını ölçekle
- **SE-Residual:** `out = SE(F(x)) + shortcut(x)`

Bu kadar. Mantık budur. Gerisi tasarım tercihi.
